In [5]:
import torch
import torch.nn.functional as F 
import matplotlib.pyplot as plt # For making figures
%matplotlib inline

In [6]:
!wget https://raw.githubusercontent.com/karpathy/makemore/master/names.txt

--2024-02-05 18:03:02--  https://raw.githubusercontent.com/karpathy/makemore/master/names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228145 (223K) [text/plain]
Saving to: 'names.txt.3'

names.txt.3         100%[===================>] 222.80K  --.-KB/s    in 0.05s   

2024-02-05 18:03:02 (4.01 MB/s) - 'names.txt.3' saved [228145/228145]



In [9]:
#read all words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:10])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']


In [10]:
# Vocab of characters & mapping to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [12]:
# Build dataset
block_size = 3 #context length: how many charactets do we take to predict the next one?
def build_dataset(words):
    X, Y = [],[]

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix] #crop and append
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X,Y

import random 
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1]) #80%
Xdev, Ydev = build_dataset(words[n1:n2]) #80%
Xte, Yte = build_dataset(words[:n2]) #80%


torch.Size([182580, 3]) torch.Size([182580])
torch.Size([22767, 3]) torch.Size([22767])
torch.Size([205347, 3]) torch.Size([205347])


In [17]:
# Utility function tn compare manual gradients PY gradients
def cmp(s, dt, t): #Comparing
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    max_dif = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact {str(ex):5s} | app {str(app):5s} | max_diff: {max_dif}')


In [18]:
# Creation of the hidden layers and the parameters 
n_embd = 10 # Embedding vector
n_hidden = 32 #Hidden neurons

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size,n_embd), generator = g)

# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator = g) * ((5/3)/(n_embd * block_size) ** 0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1 # Not needed because of batch normalization

# Layer 2 
W2 = torch.randn((n_hidden, vocab_size), generator = g) * 0.1
b2 = torch.randn((vocab_size), generator=g) * 0.1

#Batch Normalization paramaters
bngain = torch.randn((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.randn((1, n_hidden)) * 0.1

#Note: Initilization in this project is NON-standard
# because intilization sometimes starts with 
# implementation of the backward pass

parameters = [C,W1,b1,W2,b2,bngain,bnbias]
print(sum(p.nelement() for p in parameters)) #total number of parameters
for p in parameters:
    p.requires_grad = True

2217


In [20]:
batch_size = 32
#Mini batch construction
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] #batch X,Y

In [24]:
# forward pass, "chunkated" into smaller steps that are possible to backpass one at a time
emb = C[Xb] #embedding characters into vectors
embcat = emb.view(emb.shape[0], -1) #concetenate the vectors

#Linear layer 1
hprebn = embcat @ W1 + b1 #hidden layer pre-activation
#BatchNorm layer
bnmeani = 1/batch_size*hprebn
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(batch_size-1) * (bndiff2).sum(0, keepdim=True)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain  * bnraw + bnbias

